In [91]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [92]:
# data_low = pd.read_csv('../datasets/data_value_low.csv', dtype={'거래소코드':'object'}, index_col=0)
data_low = pd.read_csv('../datasets/data_value_low.csv', dtype={'거래소코드':'object'}, index_col=0)
data_low.shape

(11021, 13)

In [93]:
data_low

,year,회사명,거래소코드,회계년도,시가총액(백만원),PER(최고),PER(최저),PBR(최고),PBR(최저),PCR(최고),PCR(최저),PSR(최고),PSR(최저)
5384,2008,삼성전자(주),005930,2007/12,6.643200e+07,15.74,11.46,2.34,1.70,8.81,6.41,1.85,1.35
5385,2008,포스코홀딩스(주),005490,2007/12,3.313100e+07,18.13,6.74,2.81,1.05,13.15,4.89,3.00,1.12
5386,2008,한국전력공사(주),015760,2007/12,1.899040e+07,19.86,15.08,0.71,0.54,12.13,9.21,1.07,0.81
5387,2008,에스케이텔레콤(주),017670,2007/12,1.696949e+07,13.77,9.27,2.45,1.65,6.08,4.09,2.00,1.35
5388,2008,에이치디한국조선해양(주),009540,2007/12,1.516200e+07,24.08,5.25,8.73,1.90,10.16,2.22,2.69,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17333,2023,제이준코스메틱(주),025620,2022/12,2.426302e+04,0.00,0.00,13.29,0.00,0.00,0.00,99.19,0.00
17334,2023,(주)대유플러스,000300,2022/12,2.086506e+04,0.00,0.00,3.30,1.65,0.00,0.00,1.73,0.86
17335,2023,비케이탑스(주),030790,2022/12,1.920832e+04,0.00,0.00,11.50,0.00,0.00,0.00,21.48,0.00
17336,2023,(주)코아스,071950,2022/12,1.863705e+04,0.00,0.00,3.01,1.43,0.00,0.00,0.49,0.23


### (가치주) - 저PER, ...

- 데이터프레임명 : `data_low`

In [94]:
data_low['PER_mean'] = (data_low['PER(최고)'] + data_low['PER(최저)']) / 2
data_low['PBR_mean'] = (data_low['PBR(최고)'] + data_low['PBR(최저)']) / 2
data_low['PCR_mean'] = (data_low['PCR(최고)'] + data_low['PCR(최저)']) / 2

In [95]:
data_low = data_low[(data_low['PBR_mean'] > 0.2) | (data_low['PER_mean'] !=0) | (data_low['PCR_mean'] !=0)]

In [96]:
date_range = data_low['year'].unique().tolist()
date_range

[2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [99]:
min_df = []
for year in date_range:
    select_df = data_low[data_low['year'] == year]
    sorted_df = select_df.sort_values(by='시가총액(백만원)', ascending=False)
    # 하위 10%를 제외한 상위 90% 선택
    filter_df = sorted_df.iloc[:int(len(sorted_df) * 0.9)]

    #per 랭크 구하기
    filter_df['PER_mean_rank'] = filter_df['PER_mean'].rank(ascending=True, method='min')
    rank_per = filter_df[['PER_mean_rank', 'PER_mean', '거래소코드', '회사명', '시가총액(백만원)']].reset_index(drop=True)
    #pbr 랭크 구하기
    filter_df['PBR_mean_rank'] = filter_df['PBR_mean'].rank(ascending=True, method='min')
    rank_pbr = filter_df[['PBR_mean_rank', 'PBR_mean', '거래소코드', '회사명', '시가총액(백만원)']].reset_index(drop=True)
    #pcr 랭크 구하기
    filter_df['PCR_mean_rank'] = filter_df['PCR_mean'].rank(ascending=True, method='min')
    rank_pcr = filter_df[['PCR_mean_rank', 'PCR_mean', '거래소코드', '회사명', '시가총액(백만원)']].reset_index(drop=True)
    # print(len(rank_per))
    merge_df = pd.concat([rank_per, rank_pcr, rank_pbr], axis=1)
    merge_df = merge_df.loc[:, ~merge_df.columns.duplicated()]
    # print(merge_df)
    merge_df['total_rank'] = merge_df['PBR_mean_rank'] + merge_df['PER_mean_rank'] + merge_df['PCR_mean_rank']
    final_df = merge_df[['total_rank', '거래소코드', '시가총액(백만원)']].sort_values('total_rank').reset_index(drop=True).head(50)
    final_df['year'] = year
    final_df['weight'] = final_df['시가총액(백만원)'] / final_df['시가총액(백만원)'].sum()

    min_df.append(final_df)


In [100]:
min_df = pd.concat(min_df)

In [102]:
min_df.to_csv('../datasets/min.csv')